In [1]:
# Import required libraries
import os
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# Check device
if tf.config.list_physical_devices('GPU'):
  device = "GPU"
else:
  device = "CPU"
print(f"Using device: {device}")

Using device: CPU


In [3]:
# Define transformations
transformer = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest"
)

In [4]:
# Load data
train_path = "../images/train"
test_path = "../images/test"

train_data = transformer.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=256,
    class_mode="categorical"
)

test_data = transformer.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=256,
    class_mode="categorical"
)

Found 2741 images belonging to 3 classes.
Found 1541 images belonging to 3 classes.


In [5]:
# Get class names
class_names = list(train_data.class_indices.keys())
print(f"Classes: {class_names}")

Classes: ['paper', 'rock', 'scissors']


In [6]:
# Define model
model = Sequential()

model.add(Conv2D(12, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPool2D())
model.add(Conv2D(20, (3, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(3, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 12)      336       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 12)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 20)      2180      
                                                                 
 conv2d_2 (Conv2D)           (None, 107, 107, 32)      5792      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 89888)             0

In [7]:
# Compile model
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [8]:
# Train model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)

Epoch 1/10
11/11 [==============================] - 96s 9s/step - loss: 1.2012 - accuracy: 0.3937 - val_loss: 0.9149 - val_accuracy: 0.5769
Epoch 2/10
11/11 [==============================] - 89s 8s/step - loss: 0.8089 - accuracy: 0.6224 - val_loss: 0.7929 - val_accuracy: 0.5587
Epoch 3/10
11/11 [==============================] - 92s 8s/step - loss: 0.6812 - accuracy: 0.6614 - val_loss: 0.6165 - val_accuracy: 0.7554
Epoch 4/10
11/11 [==============================] - 93s 8s/step - loss: 0.5877 - accuracy: 0.7355 - val_loss: 0.5578 - val_accuracy: 0.7813
Epoch 5/10
11/11 [==============================] - 89s 8s/step - loss: 0.5220 - accuracy: 0.7869 - val_loss: 0.4674 - val_accuracy: 0.8060
Epoch 6/10
11/11 [==============================] - 90s 8s/step - loss: 0.4501 - accuracy: 0.8041 - val_loss: 0.4312 - val_accuracy: 0.8313
Epoch 7/10
11/11 [==============================] - 91s 8s/step - loss: 0.4170 - accuracy: 0.8318 - val_loss: 0.3973 - val_accuracy: 0.8494
Epoch 8/10
11/11 [==

In [9]:
# Evaluate model
_, test_acc = model.evaluate(test_data)
print(f"Test accuracy: {test_acc}")

7/7 [==============================] - 25s 3s/step - loss: 0.3236 - accuracy: 0.8754
Test accuracy: 0.8754056096076965


In [10]:
# Save model
model.save("model/model.h5")
